In [1]:
import pandas as pd


In [ ]:
import pandas as pd

# Load the dataset into a pandas DataFrame
file_path = '/content/test_finbert.csv'  # Replace with your file path if needed
df = pd.read_csv(file_path)

# Define the mapping function for 3 labels
def map_to_three_labels(label):
    if label in ['Bullish', 'Somewhat-Bullish']:
        return 'Positive'
    elif label in ['Bearish', 'Somewhat-Bearish']:
        return 'Negative'
    elif label == 'Neutral':
        return 'Neutral'
    else:
        return None  # Handle unexpected cases

# Apply the mapping to create a new column
df['Three_Label_Classification'] = df['Overall Sentiment Label'].apply(map_to_three_labels)

# Display the distribution of the new 3-label classification
print("\nDistribution of the new 3-label classification:")
print(df['Three_Label_Classification'].value_counts())

# Save the modified dataset with the new 3-label column
output_path = 'modified_sentiment_labels.csv'
df.to_csv(output_path, index=False)

# Confirm the saved file path
print(f"\nModified dataset saved to: {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/test_finbert.csv'

In [2]:
pip install imbalanced-learn


In [2]:
import pandas as pd

# Load the dataset (adjust file path as needed)
file_path = '/content/modified_sentiment_labels.csv'
df = pd.read_csv(file_path)
df.columns

Index(['Title', 'URL', 'Time Published', 'Authors', 'Summary', 'Banner Image',
       'Source', 'Category', 'Source Domain', 'Overall Sentiment Score',
       'Overall Sentiment Label', 'Ticker', 'Ticker Relevance Score',
       'Ticker Sentiment Score', 'Ticker Sentiment Label',
       'Three_Label_Classification'],
      dtype='object')

In [3]:


# Drop unnecessary columns
columns_to_drop = [
    'Title', 'URL', 'Time Published', 'Authors', 'Banner Image',
    'Source', 'Category', 'Source Domain', 'Overall Sentiment Score',
    'Overall Sentiment Label', 'Ticker', 'Ticker Relevance Score',
    'Ticker Sentiment Score', 'Ticker Sentiment Label'
]
df = df.drop(columns=columns_to_drop)

# Verify the remaining columns
print("Remaining columns:", df.columns)

# Check class distribution before oversampling
print("Class distribution before oversampling:")
print(df['Three_Label_Classification'].value_counts())


Remaining columns: Index(['Summary', 'Three_Label_Classification'], dtype='object')
Class distribution before oversampling:
Three_Label_Classification
Neutral     2128
Positive    1830
Negative     182
Name: count, dtype: int64


In [4]:
df.rename(columns={'Summary': 'Content', 'Three_Label_Classification': 'Sentiment'}, inplace=True)

# Verify the changes
print("Renamed columns:", df.columns)

# Check class distribution before oversampling
print("Class distribution before oversampling:")
print(df['Sentiment'].value_counts())

Renamed columns: Index(['Content', 'Sentiment'], dtype='object')
Class distribution before oversampling:
Sentiment
Neutral     2128
Positive    1830
Negative     182
Name: count, dtype: int64


In [5]:
pip install nlpaug scikit-learn imbalanced-learn


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
import nlpaug.augmenter.word as naw

In [7]:
df.columns


Index(['Content', 'Sentiment'], dtype='object')

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [9]:
import pandas as pd
from sklearn.utils import resample
import nlpaug.augmenter.word as naw



In [10]:
print("Unique values in the 'Sentiment' column:")
print(df['Sentiment'].unique())

print("\nValue counts in the 'Sentiment' column:")
print(df['Sentiment'].value_counts())


Unique values in the 'Sentiment' column:
['Negative' 'Neutral' 'Positive']

Value counts in the 'Sentiment' column:
Sentiment
Neutral     2128
Positive    1830
Negative     182
Name: count, dtype: int64


In [11]:
augmenter = naw.SynonymAug(aug_src='wordnet', aug_max=10)

# Separate classes
majority_class = df[df['Sentiment'] == 'Neutral']
minority_positive = df[df['Sentiment'] == 'Positive']
minority_negative = df[df['Sentiment'] == 'Negative']

# Check if any class is empty
if majority_class.empty:
    raise ValueError("The 'Neutral' class (majority) is empty. Check the data filtering step.")

# Augment the minority class (Negative)
augmented_negatives = [augmenter.augment(text) for text in minority_negative['Content']]
negative_augmented_df = pd.DataFrame({'Content': augmented_negatives, 'Sentiment': 'Negative'})

# Oversample the minority class (Positive)
positive_oversampled_df = resample(minority_positive,
                                   replace=True,
                                   n_samples=len(majority_class),  # Match the majority class size
                                   random_state=42)

# Oversample the minority class (Negative)
negative_oversampled_df = resample(negative_augmented_df,
                                   replace=True,
                                   n_samples=len(majority_class),  # Match the majority class size
                                   random_state=42)

# Combine all classes
balanced_df = pd.concat([majority_class, positive_oversampled_df, negative_oversampled_df], ignore_index=True)

# Verify class distribution after oversampling
print("\nClass distribution after oversampling:")
print(balanced_df['Sentiment'].value_counts())


Class distribution after oversampling:
Sentiment
Neutral     2128
Positive    2128
Negative    2128
Name: count, dtype: int64


In [12]:
# Save the balanced dataset to a CSV file
output_file_path = 'balanced_sentiment_dataset.csv'
balanced_df.to_csv(output_file_path, index=False)

print(f"Balanced dataset successfully saved to: {output_file_path}")


Balanced dataset successfully saved to: balanced_sentiment_dataset.csv
